In [ ]:
from music21 import corpus, stream
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#these are python magic commands:
%matplotlib inline 
%config InlineBackend.figure_format ='svg'

In [ ]:
schoenberg = corpus.parse('schoenberg/opus19')

### Accessing and counting things

We can figure out how to manually access all the objects from the score in this tutorial: http://web.mit.edu/music21/doc/usersGuide/usersGuide_06_stream2.html 

Specifically in the section **Accessing Scores, Parts, Measures, and Notes**

As we can see, the Score object typically contains: (1) a Metadata object, (2) a StaffGroup object, and (3) *n* Part objects. 

In [ ]:
schoenberg

In [ ]:
schoenberg[0]

In [ ]:
schoenberg[1]

In [ ]:
schoenberg[2]

(Notice the `getElementsByClass` method that is available to every type of music21 stream object. You can provide it the technical name *not* in string quotes, or else a short-hand string representing the object (e.g., "Part", "Note", etc.))

The `len` (length) basic Python command is a handy way of accessing the number of "things" in an object. This will only examine the next-lower level of the hierarchy. So inside parts are typically measures. Thus, we can access the length of the first part like so:

In [ ]:
len(schoenberg[2])

However, not every item is a measure. 

In [ ]:
len(schoenberg.getElementsByClass(stream.Part)[0].getElementsByClass(stream.Measure))

So the first part (and presumably all parts) have 9 measures. Notice that the `stream.Score` object "level" does not, technically, "contain" measures.

Also notice that it will not return an error since there is nothing inappropriate about the query:

In [ ]:
len(schoenberg.getElementsByClass('Measure'))

There is a "convenience" property for accessing parts:

In [ ]:
len(schoenberg.parts)

list comprehensions are useful when you want to print out what the actual parts are called:

In [ ]:
[i for i in schoenberg.parts]

But how can we know what else is inside the part object besides measures? We can always call `.show('text')` to print out the contents of an object container

In [ ]:
schoenberg.parts[0].show('text')

Music21 has other "convenience" calls, e.g., for getting notes: `.notes` is equivalent to `.getElementsByClass(["Note", "Chord"])`; and `.notesAndRests` is equivalent to `.getElementsByClass(['Note', 'Chord', 'Rest'])`. Also, for any class in Python, you can use the function `help(Class)` to find out what it can do.

#### ContextByClass & Context attributes
A lot of music21 objects are "smart" in that they can access properties of themselves relative to some other object. (read more here: https://web.mit.edu/music21/doc/usersGuide/usersGuide_13_music21object2.html). 

For example, `note.Note().measurenumber` allows a note to access the measure number it is located in, and `note.Note().beat` or `note.Note().beatStrength` will return the note's beat position or beat strength assuming there is a time signature object.

In this case the "context" refers to the next-closest hierarchy object. Since there can be multiple *types* of context, there is a whole method for this: `getContextByClass()` 

We can use `getContextByClass()` for retrieving things like what part a note is in (`getContextByClass('Part')`) or what key a particular note is in (`getContextByClass('Key')`) -- assuming the work has at least one part object and one key object.

## Multiple Methods: Counting Durations

Our goal is to count up all the rhythmic durations. But, how are we defining "one unit"? Are any two or more notes that sound together counted as "one" duration or two? What about across the parts? Let's try a few different methods to see what we can do. 

#### Method one:
count notes in each part (i.e., hand) separately, except when they form a "chord" object (i.e., are a simultaneous event within the part) where we will count any simultaneities as a single object with a single duration. Fortunately "chords" are counted as "notes"

In [ ]:
durs = []
for i in schoenberg.parts:
    for m in i.getElementsByClass(stream.Measure):
         for n in m.notes:
             print(n, n.duration.quarterLength, n.activeSite)
             durs.append(n.duration.quarterLength)

In [ ]:
vals, counts = np.unique(durs, return_counts=True)
print(vals, counts)

Let's graph the totals:

In [ ]:
sns.set(style="darkgrid")
sns.barplot(vals,counts,color="skyblue");

#### Method two:
use music21's `score.flat` property to "flatten" the hierarchical encoding into a single stream, then count note durations. Note by doing this, we no longer have access to the part where the original chord or note appeared.

In [ ]:
durs2 = []
for n in schoenberg.flat.notes:
    print(n, n.duration.quarterLength, n.activeSite)
    durs2.append(n.duration.quarterLength)

In [ ]:
vals2, counts2 = np.unique(durs2, return_counts=True)
print(vals2, counts2)

In [ ]:
# sns.set(style="darkgrid")
sns.barplot(vals2,counts2,color="skyblue");

This time we counted 4 more 8th notes,1 more quarter note, and 3 more half notes! Why? What was missed? (Notice the method you choose can impact the outcome. This is just for a single piece. Imagine the difference across a whole corpus?)

Our second method *should* work in the same way as using `score.recurse()`. Let's check a third method. Notice using `score.recurse()` allows us to retrieve the original object's location.

#### Method 3

In [ ]:
durs3 = []
for n in schoenberg.recurse().notes:
    print(n, n.activeSite, n.duration.quarterLength)
    durs3.append(n.duration.quarterLength)

In [ ]:
vals3, counts3 = np.unique(durs3, return_counts=True)
print(vals3, counts3)

OK, this method got us the same as the counts for method #2 (no need to plot).  

So why did our first method differ from the last two methods? Why did our count of eighth notes (and quarter and half notes) increase?  

Let's try tallying events manually. Be sure to count tied notes according to their total durations. "LHRH" should only count events that happen in both hands at the same time as a single object with single duration. "Total" should ignore "LHRH" and sum the LH and RH totals. 

Fill in the following table:

|   | 0.25 | 0.5  | 1.  | 1.5  | 2  | 2.5 |
|---|---|---|---|---|---|---|
|  RH |  |   |   |   |   |   |
|  LH |   |   |   |   |   |   |
|  LHRH |   |   |   |   |   |   |
| Total  |   |   |   |   |   |   |

In [ ]:
schoenberg.show()

Q: Do your counts from any row match the counts from any of our methods? What do you think might have happened? (For instance, why are there no counts of 2.5 or half+eighth?

In [ ]:
schoenberg.show()

Let's try one final method, using the `stripTies()` method

#### Method 4

In [ ]:
durs4 = []
for i in schoenberg.parts:
    for n in i.getElementsByClass(stream.Measure).stripTies():
        if not n.isRest:
            print(n, n.duration.quarterLength, n.activeSite)
            durs4.append(n.duration.quarterLength)

In [ ]:
vals4, counts4 = np.unique(durs4, return_counts=True)
print(vals4, counts4)

In [ ]:
sns.barplot(vals4, counts4, color="skyblue")

Notice that **method one** did not find elements that were in sub-streams (`stream.Voice`) of the main Stream. But **methods one through three** did not properly count notes that were tied across measures. We had to use `stripTies()` at the part level, which by default removes measure objects (and doesn't work properly if we try to retain them) to get the correct count.

Notice that there are other methods still that will obtain the incorrect count given what we want. **Hopefully this excercise illustrates the importance of "sanity checks" on a very small sample.** That is, don't **assume** that a piece of code does what you think it does.

As a final illustration, below is what we get when we rely on music21's built-in `music21.graph` object for counting durations. Because scores can be assembled many different ways, and there are many different ways to access the objects in music21, a "one size fits all" approach often fails. Be sure to check that you are getting the data you think you need!

In [ ]:
p = graph.plot.HistogramQuarterLength(schoenberg);
p.run()